# Interactions - Lab

## Introduction

In this lab, you'll explore interactions in the Ames Housing dataset.

## Objectives

You will be able to:
- Implement interaction terms in Python using the `sklearn` and `statsmodels` packages 
- Interpret interaction variables in the context of a real-world problem 

## Build a baseline model 

You'll use a couple of built-in functions, which we imported for you below: 

In [1]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

If you still want to build a model in the end, you can do that, but this lab will just focus on finding meaningful insights in interactions and how they can improve $R^2$ values.

In [2]:
regression = LinearRegression()

Create a baseline model which includes all the variables we selected from the Ames housing data set to predict the house prices. Then use 10-fold cross-validation and report the mean $R^2$ value as the baseline $R^2$.

In [3]:
ames = pd.read_csv('ames.csv')

continuous = ['LotArea', '1stFlrSF', 'GrLivArea', 'SalePrice']
categoricals = ['BldgType', 'KitchenQual', 'SaleType', 'MSZoning', 'Street', 'Neighborhood']

In [4]:
# continuous variables
ames_cont = ames[continuous]
ames_log = np.log(ames_cont)
log_names = [f'{feature}_log' for feature in continuous]
ames_log.columns = log_names
ames_log.head()

,LotArea_log,1stFlrSF_log,GrLivArea_log,SalePrice_log
0,9.041922,6.752270,7.444249,12.247694
1,9.169518,7.140453,7.140453,12.109011
2,9.328123,6.824374,7.487734,12.317167
3,9.164296,6.867974,7.448334,11.849398
4,9.565214,7.043160,7.695303,12.429216


In [5]:
# helper function to normalize
def normalize(feature):
    return (feature - feature.mean()) / feature.std()

In [6]:
# normalize ames_log
ames_log_norm = ames_log.apply(normalize)

In [7]:
# one hot encode categoricals
ames_ohe = pd.get_dummies(ames[categoricals], prefix=categoricals, drop_first=True)

In [21]:
# concat continuous with categorical
# preprocessed = pd.concat([ames_log_norm, ames_ohe], axis=1)
# X = preprocessed.drop(columns=['SalePrice_log'])
# y = preprocessed['SalePrice_log']

# -----uncomment above to run with log normalized continuous variables----
# I changed the below to match solution to ensure rest of lab was correct


# the solution does not append log normalized (i believe this is an error)
preprocessed = pd.concat([ames_cont, ames_ohe], axis=1)
X = preprocessed.drop(columns=['SalePrice'])
y = preprocessed['SalePrice']


In [22]:
#get baseline model
regression = LinearRegression()
crossvalidation = KFold(n_splits=10, shuffle=True, random_state=1)
baseline = np.mean(cross_val_score(regression, X, y, cv=crossvalidation))
baseline

0.7773817128042101

## See how interactions improve your baseline

Next, create all possible combinations of interactions, loop over them and add them to the baseline model one by one to see how they affect the $R^2$. We'll look at the 3 interactions which have the biggest effect on our $R^2$, so print out the top 3 combinations.

You will create a `for` loop to loop through all the combinations of 2 predictors. You can use `combinations` from itertools to create a list of all the pairwise combinations. To find more info on how this is done, have a look [here](https://docs.python.org/2/library/itertools.html).

Since there are so many different neighbourhoods we will exclude

In [11]:
from itertools import combinations

In [12]:
features = X.columns
features

Index(['LotArea', '1stFlrSF', 'GrLivArea', 'BldgType_2fmCon',
       'BldgType_Duplex', 'BldgType_Twnhs', 'BldgType_TwnhsE',
       'KitchenQual_Fa', 'KitchenQual_Gd', 'KitchenQual_TA', 'SaleType_CWD',
       'SaleType_Con', 'SaleType_ConLD', 'SaleType_ConLI', 'SaleType_ConLw',
       'SaleType_New', 'SaleType_Oth', 'SaleType_WD', 'MSZoning_FV',
       'MSZoning_RH', 'MSZoning_RL', 'MSZoning_RM', 'Street_Pave',
       'Neighborhood_Blueste', 'Neighborhood_BrDale', 'Neighborhood_BrkSide',
       'Neighborhood_ClearCr', 'Neighborhood_CollgCr', 'Neighborhood_Crawfor',
       'Neighborhood_Edwards', 'Neighborhood_Gilbert', 'Neighborhood_IDOTRR',
       'Neighborhood_MeadowV', 'Neighborhood_Mitchel', 'Neighborhood_NAmes',
       'Neighborhood_NPkVill', 'Neighborhood_NWAmes', 'Neighborhood_NoRidge',
       'Neighborhood_NridgHt', 'Neighborhood_OldTown', 'Neighborhood_SWISU',
       'Neighborhood_Sawyer', 'Neighborhood_SawyerW', 'Neighborhood_Somerst',
       'Neighborhood_StoneBr', 'Neighbor

In [13]:
combos = combinations(features, 2)
all_interactions = [combo for combo in combos]

In [16]:
# store interaction results in dictionary
interactions = {}

for i, interaction in enumerate(all_interactions):
    X_interact = X.copy()
    X_interact[interaction] = X[interaction[0]] * X[interaction[1]]
    r_squared = np.mean(cross_val_score(regression, X_interact, y, scoring='r2', cv=crossvalidation))
    
    if (r_squared > baseline):
        interactions[interaction] = r_squared
        print(interaction, r_squared)

('LotArea', 'BldgType_2fmCon') 0.778589913335075
('LotArea', 'SaleType_Con') 0.7779506251409576
('LotArea', 'SaleType_ConLI') 0.7774876855027641
('LotArea', 'SaleType_New') 0.7931273923137889
('LotArea', 'SaleType_Oth') 0.7780175799522856
('LotArea', 'SaleType_WD') 0.7906354461190374
('LotArea', 'Street_Pave') 0.7780427534958254
('LotArea', 'Neighborhood_BrkSide') 0.7784720543771675
('LotArea', 'Neighborhood_Edwards') 0.8093314939295133
('LotArea', 'Neighborhood_Mitchel') 0.7783501384312512
('LotArea', 'Neighborhood_NoRidge') 0.778376735698773
('LotArea', 'Neighborhood_NridgHt') 0.7837853111904158
('LotArea', 'Neighborhood_OldTown') 0.778059635794865
('LotArea', 'Neighborhood_Sawyer') 0.7774192053514778
('LotArea', 'Neighborhood_SawyerW') 0.7780438658294352
('1stFlrSF', 'BldgType_2fmCon') 0.7777232679880279
('1stFlrSF', 'BldgType_Duplex') 0.778838938583214
('1stFlrSF', 'KitchenQual_Gd') 0.7822396599316044
('1stFlrSF', 'SaleType_Con') 0.7786512786632713
('1stFlrSF', 'MSZoning_FV') 0.777

('BldgType_TwnhsE', 'SaleType_ConLw') 0.7773817128042102
('BldgType_TwnhsE', 'SaleType_Oth') 0.7773817128042102
('BldgType_TwnhsE', 'SaleType_WD') 0.7787074225880075
('BldgType_TwnhsE', 'MSZoning_FV') 0.7774783415504497
('BldgType_TwnhsE', 'MSZoning_RH') 0.7783157402388724
('BldgType_TwnhsE', 'MSZoning_RL') 0.7797751983025766
('BldgType_TwnhsE', 'MSZoning_RM') 0.777933836684616
('BldgType_TwnhsE', 'Street_Pave') 0.7773817128043155
('BldgType_TwnhsE', 'Neighborhood_Blueste') 0.777410318031088
('BldgType_TwnhsE', 'Neighborhood_BrDale') 0.7774112675533971
('BldgType_TwnhsE', 'Neighborhood_BrkSide') 0.7773817128042102
('BldgType_TwnhsE', 'Neighborhood_ClearCr') 0.7773817128042102
('BldgType_TwnhsE', 'Neighborhood_Crawfor') 0.7802922446899256
('BldgType_TwnhsE', 'Neighborhood_Edwards') 0.7776822137983636
('BldgType_TwnhsE', 'Neighborhood_Gilbert') 0.7773817128042102
('BldgType_TwnhsE', 'Neighborhood_IDOTRR') 0.7773817128042102
('BldgType_TwnhsE', 'Neighborhood_Mitchel') 0.777483954381685
('

('SaleType_Con', 'Neighborhood_SWISU') 0.7773817128042102
('SaleType_Con', 'Neighborhood_Sawyer') 0.7773817128042102
('SaleType_Con', 'Neighborhood_SawyerW') 0.7773817128042102
('SaleType_Con', 'Neighborhood_StoneBr') 0.7773817128042102
('SaleType_Con', 'Neighborhood_Timber') 0.7773817128042102
('SaleType_Con', 'Neighborhood_Veenker') 0.7782773990739428
('SaleType_ConLD', 'SaleType_ConLI') 0.7773817128042102
('SaleType_ConLD', 'SaleType_ConLw') 0.7773817128042102
('SaleType_ConLD', 'SaleType_New') 0.7773817128042102
('SaleType_ConLD', 'SaleType_Oth') 0.7773817128042102
('SaleType_ConLD', 'SaleType_WD') 0.7773817128042102
('SaleType_ConLD', 'MSZoning_FV') 0.7773817128042102
('SaleType_ConLD', 'MSZoning_RH') 0.7773817128042102
('SaleType_ConLD', 'Neighborhood_Blueste') 0.7773817128042102
('SaleType_ConLD', 'Neighborhood_BrDale') 0.7773817128042102
('SaleType_ConLD', 'Neighborhood_BrkSide') 0.7773817128042102
('SaleType_ConLD', 'Neighborhood_ClearCr') 0.7773817128042102
('SaleType_ConLD',

('SaleType_WD', 'Neighborhood_NWAmes') 0.7774241504421342
('SaleType_WD', 'Neighborhood_NoRidge') 0.7773817128043123
('SaleType_WD', 'Neighborhood_NridgHt') 0.7792397274534677
('SaleType_WD', 'Neighborhood_SWISU') 0.7773817128043146
('SaleType_WD', 'Neighborhood_StoneBr') 0.7794615136331517
('SaleType_WD', 'Neighborhood_Veenker') 0.7778534178935173
('MSZoning_FV', 'MSZoning_RH') 0.7773817128042102
('MSZoning_FV', 'MSZoning_RL') 0.7773817128042102
('MSZoning_FV', 'MSZoning_RM') 0.7773817128042102
('MSZoning_FV', 'Street_Pave') 0.777381712804328
('MSZoning_FV', 'Neighborhood_Blueste') 0.7773817128042102
('MSZoning_FV', 'Neighborhood_BrDale') 0.7773817128042102
('MSZoning_FV', 'Neighborhood_BrkSide') 0.7773817128042102
('MSZoning_FV', 'Neighborhood_ClearCr') 0.7773817128042102
('MSZoning_FV', 'Neighborhood_CollgCr') 0.7773817128042102
('MSZoning_FV', 'Neighborhood_Crawfor') 0.7773817128042102
('MSZoning_FV', 'Neighborhood_Edwards') 0.7773817128042102
('MSZoning_FV', 'Neighborhood_Gilbert'

('Neighborhood_Blueste', 'Neighborhood_Veenker') 0.7773817128042102
('Neighborhood_BrDale', 'Neighborhood_BrkSide') 0.7773817128042102
('Neighborhood_BrDale', 'Neighborhood_ClearCr') 0.7773817128042102
('Neighborhood_BrDale', 'Neighborhood_CollgCr') 0.7773817128042102
('Neighborhood_BrDale', 'Neighborhood_Crawfor') 0.7773817128042102
('Neighborhood_BrDale', 'Neighborhood_Edwards') 0.7773817128042102
('Neighborhood_BrDale', 'Neighborhood_Gilbert') 0.7773817128042102
('Neighborhood_BrDale', 'Neighborhood_IDOTRR') 0.7773817128042102
('Neighborhood_BrDale', 'Neighborhood_MeadowV') 0.7773817128042102
('Neighborhood_BrDale', 'Neighborhood_Mitchel') 0.7773817128042102
('Neighborhood_BrDale', 'Neighborhood_NAmes') 0.7773817128042102
('Neighborhood_BrDale', 'Neighborhood_NPkVill') 0.7773817128042102
('Neighborhood_BrDale', 'Neighborhood_NWAmes') 0.7773817128042102
('Neighborhood_BrDale', 'Neighborhood_NoRidge') 0.7773817128042102
('Neighborhood_BrDale', 'Neighborhood_NridgHt') 0.777381712804210

('Neighborhood_Gilbert', 'Neighborhood_NridgHt') 0.7773817128042102
('Neighborhood_Gilbert', 'Neighborhood_OldTown') 0.7773817128042102
('Neighborhood_Gilbert', 'Neighborhood_SWISU') 0.7773817128042102
('Neighborhood_Gilbert', 'Neighborhood_Sawyer') 0.7773817128042102
('Neighborhood_Gilbert', 'Neighborhood_SawyerW') 0.7773817128042102
('Neighborhood_Gilbert', 'Neighborhood_Somerst') 0.7773817128042102
('Neighborhood_Gilbert', 'Neighborhood_StoneBr') 0.7773817128042102
('Neighborhood_Gilbert', 'Neighborhood_Timber') 0.7773817128042102
('Neighborhood_Gilbert', 'Neighborhood_Veenker') 0.7773817128042102
('Neighborhood_IDOTRR', 'Neighborhood_MeadowV') 0.7773817128042102
('Neighborhood_IDOTRR', 'Neighborhood_Mitchel') 0.7773817128042102
('Neighborhood_IDOTRR', 'Neighborhood_NAmes') 0.7773817128042102
('Neighborhood_IDOTRR', 'Neighborhood_NPkVill') 0.7773817128042102
('Neighborhood_IDOTRR', 'Neighborhood_NWAmes') 0.7773817128042102
('Neighborhood_IDOTRR', 'Neighborhood_NoRidge') 0.7773817128

('Neighborhood_Somerst', 'Neighborhood_Veenker') 0.7773817128042102
('Neighborhood_StoneBr', 'Neighborhood_Timber') 0.7773817128042102
('Neighborhood_StoneBr', 'Neighborhood_Veenker') 0.7773817128042102
('Neighborhood_Timber', 'Neighborhood_Veenker') 0.7773817128042102


In [19]:
sorted_dict = {k: v for k, v in sorted(interactions.items(), key=lambda item: item[1], reverse=True)}

In [20]:
sorted_dict

{('LotArea', 'Neighborhood_Edwards'): 0.8093314939295133,
 ('GrLivArea', 'Neighborhood_Edwards'): 0.8077114235219648,
 ('1stFlrSF', 'Neighborhood_Edwards'): 0.8033414554231054,
 ('LotArea', 'SaleType_New'): 0.7931273923137889,
 ('GrLivArea', 'Neighborhood_NoRidge'): 0.7930790499039408,
 ('LotArea', 'SaleType_WD'): 0.7906354461190374,
 ('1stFlrSF', 'Neighborhood_NridgHt'): 0.7845792550853155,
 ('LotArea', 'Neighborhood_NridgHt'): 0.7837853111904158,
 ('GrLivArea', 'Neighborhood_NridgHt'): 0.7830123744526247,
 ('SaleType_New', 'Neighborhood_Edwards'): 0.7829627554318175,
 ('1stFlrSF', 'Neighborhood_NoRidge'): 0.7826567131582727,
 ('BldgType_TwnhsE', 'Neighborhood_StoneBr'): 0.782246164763044,
 ('1stFlrSF', 'KitchenQual_Gd'): 0.7822396599316044,
 ('GrLivArea', 'KitchenQual_Gd'): 0.7819495257069627,
 ('1stFlrSF', 'Neighborhood_Veenker'): 0.7811744839803141,
 ('1stFlrSF', 'Neighborhood_StoneBr'): 0.780679111514352,
 ('SaleType_New', 'Neighborhood_StoneBr'): 0.780611140596343,
 ('BldgType_Tw

It looks like the top interactions involve the Neighborhood_Edwards feature so lets add the interaction between LotArea and Edwards to our model.

We can interpret this feature as the relationship between LotArea and SalePrice when the house is in Edwards or not.

## Visualize the Interaction

Separate all houses that are located in Edwards and those that are not. Run a linear regression on each population against `SalePrice`. Visualize the regression line and data points with price on the y axis and LotArea on the x axis.

## Build a final model with interactions

Use 10-fold cross-validation to build a model using the above interaction. 

In [ ]:
# code here

Our $R^2$ has increased considerably! Let's have a look in `statsmodels` to see if this interactions are significant.

In [ ]:
# code here

What is your conclusion here?

In [ ]:
# formulate your conclusion

## Summary

You should now understand how to include interaction effects in your model! As you can see, interactions can have a strong impact on linear regression models, and they should always be considered when you are constructing your models.